In [22]:
import os
import pandas as pd
import numpy as np

# Change the working directory to strategy_lab
os.chdir("/Users/sugang/Desktop/projects/2024coin/strategy_lab")

# Now, you can read the CSV file without specifying the full path
df = pd.read_csv("1/full_data_with_predictions.csv")
df

,time_utc,open,high,low,close,volume_krw,volume_market,Daily Return,predict_close
0,2017-10-01,4889000.0,4978000.0,4682000.0,4962000.0,1.331325e+08,27.323332,0.013687,4303650.5
1,2017-10-02,4962000.0,5095000.0,4956000.0,5025000.0,1.599315e+08,31.728004,0.012696,4412025.5
2,2017-10-03,5021000.0,5079000.0,4811000.0,4964000.0,5.890130e+07,11.899307,-0.012139,4474568.0
3,2017-10-04,4987000.0,5002000.0,4837000.0,4895000.0,3.458813e+07,7.038744,-0.013900,4504902.5
4,2017-10-05,4894000.0,4993000.0,4811000.0,4926000.0,2.637132e+07,5.363652,0.006333,4504320.0
...,...,...,...,...,...,...,...,...,...
2596,2024-11-09,106845000.0,106990000.0,105873000.0,106294000.0,2.876788e+11,2702.452888,-0.005138,98971560.0
2597,2024-11-10,106294000.0,112470000.0,106050000.0,111440000.0,8.895481e+11,8111.036561,0.048413,101134830.0
2598,2024-11-11,111440000.0,125073000.0,111273000.0,124503000.0,1.413171e+12,12056.831324,0.117220,103599640.0
2599,2024-11-12,124476000.0,128010000.0,118864000.0,124967000.0,2.115376e+12,17037.759563,0.003727,107227736.0


In [23]:
from handle_candle import resample_df
from datetime import datetime
from performance_utils import get_performance

def vol_b_o_performance(k, hh, df=df):
    # execution_time makes a lot of diff.
    df_copy = df.copy()
    execution_time = datetime.strptime(f"{hh}:00", "%H:%M")
    df_copy = resample_df(df_copy, execution_time=execution_time)
    df_copy["t-1_range"] = df_copy["high"] - df_copy["low"]
    df_copy["t-1_range"] = df_copy["t-1_range"].shift(1)
    df_copy["target"] = df_copy["open"] + df_copy["t-1_range"] * k

    # >, >= check it leads big diff.
    df_copy["signal"] = np.where(df_copy["high"] > df_copy["target"], 1, 0)
    df_copy["strategy_returns2"] = df_copy["signal"] * ((df_copy["close"] * 0.998 - df_copy["target"] * 1.002) / df_copy["target"] * 1.002)
    df_copy["cumulative_returns2"] = (1 + df_copy["strategy_returns2"]).cumprod()

    
    print("--base--")
    print(f"result of k: {k}, time: {hh}")
    print(get_performance(df_copy))

def vol_b_o_pred_performance(k, hh, df=df):
    # execution_time makes a lot of diff.
    df_copy = df.copy()
    # execution_time = datetime.strptime(f"{hh}:00", "%H:%M")
    # df_copy = resample_df(df_copy, execution_time=execution_time)
    df_copy["t-1_range"] = df_copy["high"] - df_copy["low"]
    df_copy["t-1_range"] = df_copy["t-1_range"].shift(1)
    df_copy["target"] = df_copy["open"] + df_copy["t-1_range"] * k

    # >, >= check it leads big diff.
    if "predict_close" in df_copy.columns:
        df_copy["signal"] = np.where((df_copy["high"] > df_copy["target"]) & (df_copy["predict_close"] > df_copy["target"]),1,0)
    else:
        print(df_copy.columns)
        return "FUCk"
    df_copy["strategy_returns2"] = df_copy["signal"] * ((df_copy["close"] * 0.998 - df_copy["target"] * 1.002) / df_copy["target"] * 1.002)
    df_copy["cumulative_returns2"] = (1 + df_copy["strategy_returns2"]).cumprod()

    
    print("--base--")
    print(f"result of k: {k}, time: {hh}")
    print(get_performance(df_copy))

In [24]:
vol_b_o_performance(k=0.7, hh=00)
vol_b_o_pred_performance(k=0.7, hh=00)

--base--
result of k: 0.7, time: 0
{'total_return': 1.7167921751064394, 'cagr': 0.15062774852367, 'mdd': 0.6358496594061857, 'win_rate': None, 'gain_loss_ratio': None, 'holding_time_ratio': None, 'investing_period': 2600}
Index(['time_utc', 'open', 'high', 'low', 'close', 'volume_krw',
       'volume_market', 'Daily Return', 'predict_close'],
      dtype='object')
--base--
result of k: 0.7, time: 0
{'total_return': -0.3208652251344404, 'cagr': -0.05287084969849187, 'mdd': 0.4683132067195151, 'win_rate': None, 'gain_loss_ratio': None, 'holding_time_ratio': None, 'investing_period': 2600}
